<a href="https://colab.research.google.com/github/dvschultz/MUNIT/blob/master/MUNIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MUNIT
MUNIT stands for Multimodal UNsupervised Image-to-image Translation. That’s a lot of words to say it can convert images of cats to images of dogs, or images of horses to images of zebras. Basically, it learns to convert one set of images (a “domain”) to another set.

[Original repo](https://github.com/NVlabs/MUNIT) | [Forked repo](https://github.com/dvschultz/MUNIT)

There is also a new version of MUNIT in the Imaginaire library. I personally didn’t like it as much but YMMV.

---



If you find this notebook useful, consider signing up for my [Patreon](https://www.patreon.com/bustbright) or [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join). You can also send me a one-time payment on [Venmo](https://venmo.com/Derrick-Schultz).

## Setup

First let’s check to see what GPU we’ve been assigned.

In [ ]:
!nvidia-smi -L

Next let’s connect our Google Drive account to Colab. This is optional but highly recommended.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Install repo

The nnext cell will install the MUNIT repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [ ]:
import os

if os.path.isdir("/content/drive/MyDrive/colab-munit/MUNIT/"):
    %cd /content/drive/MyDrive/colab-munit/MUNIT/
elif os.path.isdir("/content/drive/"):
    !mkdir /content/drive/MyDrive/colab-munit
    %cd /content/drive/MyDrive/colab-munit
    !git clone https://github.com/dvschultz/MUNIT
    %cd MUNIT
else:
    print('Drive not mounted, installing in /content/')  
    !git clone https://github.com/dvschultz/MUNIT
    %cd MUNIT

#install dependencies
!pip install tensorboardX torchfile

##Dataset Preparation

MUNIT requires two different sets of images. Unlike Pix2Pix or similar models, these two sets of images do not need to be tightly coupled (for example: a color image and its grayscale equivalent). The images also do not need to be the same size.

I recommend a minimum of ~250 images per folder. This will allow for the model to generalize fairly well (the more images, the better). 

Once you have created the images you will need to upload them to your server. The folder structure should look like this:
* Top level folder name (I usually use X2Y, where X is the first dataset and Y is the second.)
* ├ trainA (training set from X)
* ├ trainB (training set from Y)
* ├ testA (testing set from X)
* ├ testB (testing set from Y)

For our purposes `trainA`/`testA` and `trainB`/`testB` can be the same, but in a "scientific" setting you would want these folder to contain different images.

There are two ways to upload the images.
1. Zip up the folder and upload it to Colab in the `MUNIT/datasets` folder. This is probably slow and depending how large your dataset is can use up a lot of the Disk space in Colab.
2. You can connect your Google Drive to Colab and host your files in Google Drive (This does not require you zipping up your folder, but does require a Google Drive account). This option needs a little setting up, but it saves you disk space and time once it’s set up.


#Training
Once you have your dataset prepped and uploaded, the last thing to do is create a YAML config file. Let’s first duplicate one of the example files. In the code below I recommend renaming the file to match the top level folder name you used when uploading the dataset. This helps keep track of file naming and will remind you what settings you used should you need to retrain the model ever again.

In [ ]:
!cp ./configs/edges2shoes_folder.yaml ./configs/myprojectname_folder.yaml

Next, double-click on the .yaml file in the File viewer to your left and edit the last few lines of the file. 

* `data_root`: should point to the path of your dataset
* `new_size`, `crop_image_height`, `crop_image_width` (optional): I recommend setting this to a larger size, I find it leads to better training (I usually use `1024`, `512`, and `512` respectively).

Ok, now it’s time to train. We can see all of the options for training by running the next cell.

In [ ]:
!python train.py --help

In my experience all we really need to worry about it the `--config` argument.

In [ ]:
!python train.py --config ./configs/myprojectname_folder.yaml

/content/MUNIT


### Resume Training

If Colab shuts off before you complete training, you can resume training with this command:

In [ ]:
!python train.py --config ./configs/myprojectname_folder.yaml --resume

#Testing
If you’re only using Colab for testing, you’ll need to do the following:
1. Move your config .yaml from training into the `configs` folder.
2. Create a `models` folder and put the generator .pt file in there (the generator file begins with `gen_`)
3. Create an `inputs` folder and a `styles` folder. You’ll put images you want to translate in the `inputs` folder and images you want to guide style in the `styles` folder (See the Guided Translation).

In [ ]:
#make some folders (only run once)
%mkdir models
%mkdir styles

In [ ]:
#testing with a single image
#this generates 10 random images from the model
!python test.py --config /content/MUNIT/configs/512-sm_birds2floralmag_folder.yaml --input /content/MUNIT/inputs/birdsAustraliav5Goul_0208.png --output_folder outputs --checkpoint /content/MUNIT/models/gen_00240000.pt --a2b 1

In [ ]:
!zip -r outputs.zip ./outputs

updating: outputs/ (stored 0%)
updating: outputs/output004.jpg (deflated 1%)
updating: outputs/output006.jpg (deflated 3%)
updating: outputs/output002.jpg (deflated 1%)
updating: outputs/output009.jpg (deflated 1%)
updating: outputs/output007.jpg (deflated 1%)
updating: outputs/output005.jpg (deflated 2%)
updating: outputs/output000.jpg (deflated 3%)
updating: outputs/output001.jpg (deflated 1%)
updating: outputs/output003.jpg (deflated 1%)
updating: outputs/output008.jpg (deflated 0%)
updating: outputs/input.jpg (deflated 1%)
updating: outputs/output069.jpg (deflated 2%)
updating: outputs/output077.jpg (deflated 2%)
updating: outputs/output091.jpg (deflated 1%)
updating: outputs/output037.jpg (deflated 1%)
updating: outputs/output032.jpg (deflated 2%)
updating: outputs/output065.jpg (deflated 3%)
updating: outputs/output087.jpg (deflated 1%)
updating: outputs/output013.jpg (deflated 1%)
updating: outputs/output088.jpg (deflated 1%)
updating: outputs/output040.jpg (deflated 1%)
updatin

##Testing Options

`--num_style`
How many "styles" you want to output

`--num_style_start` What index to start with. If you combine this with `--num_style 1` you can generate only the style you want once determined (for example, run 500 samples, pick your favorite, then only ever test with that one.)

In [ ]:
#testing with a single image, use --num_styles to set the number of styles
!rm outputs/*.*
!python test.py --config /content/MUNIT/configs/512-sm_birds2floralmag_folder.yaml --input /content/MUNIT/inputs/birdsAustraliav5Goul_0208.png --output_folder outputs --checkpoint /content/MUNIT/models/gen_00240000.pt --a2b 1 --num_style 1 --num_style_start 65

rm: cannot remove 'outputs/*.*': No such file or directory


##Guided Translation
MUNIT also allows you to do what they call "guided translation." This allows you to pass it an image and use that image as the "style" (for lack of a better term) for the output image.

In [ ]:
#guided translation
python test.py --config configs/edges2shoes_folder.yaml --input inputs/edges2shoes_edge.jpg --output_folder results --checkpoint models/edges2shoes.pt --a2b 1 --style inputs/edges2shoes_shoe.jpg

##Batch Testing
Both of the above examples only generate samples from one image. If you want to test it on a whole folder of images you use the `test_batch.py` script.

Note: you can’t currently batch test with a guided translation image. (I’m working on it.)

In [ ]:
python test_batch.py --config configs/edges2shoes_folder.yaml --input_folder inputs --output_folder outputs --checkpoint models/edges2shoes.pt --a2b 1

In [ ]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/dvschultz/MUNIT
   8b23e8f..e46453b  master     -> origin/master
Updating 8b23e8f..e46453b
Fast-forward
 test.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)
